# S&P 2019 Data Wrangling

**by Marc Angelo Acebedo**

## Table of Contents
- [Introduction](#intro)
- [Data Wrangling](#wrangling)
    - [Gather](#gather)
    - [Assess](#assess)
    - [Clean](#clean)
- [Store](#store)

## I) Introduction <a id = "intro"></a>

**Broad question:** How do price forecasts for each firm in the S&P 2019 Index compare to their corresponding actual prices?

**Approach:** Analyze difference in means between average forecast EPS and average actual EPS for each firm.

I will be analyzing quarterly price returns within the past 20 years for the firms present in the S&P 500 2019 Index.

> At first, I wanted to analyze the forecasted vs. actual price earnings of the S&P in its entirety for the past 20 years. However, considering that firms continuously enter and leave stock indices every year, there would be varying levels of inconsistencies and marginal errors when comparing annual S&P returns alone. To combat this problem, I have isolated these two approaches:
- Analyze the historical earnings of *only* the firms present in the S&P 2019 Index
- Keep track of all firms that were present in the S&P for the past 20 years. Keep track of how many times each firm appeared in the Index and for those with the least count, analyze them individually on how they differ from the firms that stayed for longer.

Here is a breakdown of my final clean CSVs' features, `features.csv` and `avgs.csv`.

### `features.csv`

- **firm_id**
- **feature**
- **date**
- **term**
- **value**

### `avgs.csv`

- **firm_id**
- **average**
- **average_type**
- **time_period**
- **feature**

> The **firm_id** values refer to their corresponding foreign keys in `firms.csv`. Further explanation of each column can be found in the `data_exploratory.ipynb` file.

## II) Data Wrangling <a id="wrangling"></a>

To gather the data depicted under the `./data` folder, I used Bloomberg Excel functions.

### A) Gather <a id = "gather"></a>
> **APPROACH 1:** Focus on the firms that appear in the 2019 S&P Index and analyze their forecasted vs. actual price earnings for the last 20 years.

To ensure consistency in analysis among multiple firms, I divide both the forecasted and actual EPS terms by *fiscal period* instead of calendar period. On the other hand, EOD Price and Forecasted EPS (3 months prior) have both been recorded by *calendar period*, so this is a discrepancy we must be aware of in our analysis. 

#### Through the Bloomberg Excel functions, I gathered four datasets with different purposes:

- historical forecasted EPS
- historical actual EPS
- historical actual EOD price
- historical forecasted EPS relying on terms

---
Before delving into the data, let's define what above terms:

**EPS**

> EPS stands for ***Estimated Price Earnings.*** The formal definition of EPS given by Investopedia is this:

Earnings per share is the portion of a company's profit that is allocated to each outstanding share of a common stock, serving as an indicator of the company's financial health.

In other words, the EPS is a portion of the company's **net income** after all of their dividends are paid off. Dividends are profits that are paid out to shareholders of the company. EPS is one of the most useful and valuable financial measurements because they ***determine a stock's worth.*** The higher the stock, the more the company can pay out dividends to its shareholders, and the more net profit they are determined to generate.

$$ EPS = \frac{Net Income - Preferred Dividends}{Weighted Average Common Shares Outstanding}\$$

**EOD**

> EOD stands for the ***End of Day*** price. For any given day, the EOD marks the ***price at which the stock was valued*** at the end of the day's trading period.

In [11]:
PATH = './data/'
PATH_CLEAN = './data/clean/'
PATH_CLEAN_AVGS = './data/clean/averages/'
PATH_CLEAN_AVGS_SUB = './data/clean/averages/components/'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

from functools import reduce

### Import Data <a id = "import"></a>

### Let's summarize the contents of the following DataFrames as we import them.
> All DataFrames consist of the 505 firms found in the 2019 S&P Index with EPS and EOD data encompassing 20 years: from January 1999 until December 2019.

**Historic forecasted EPS**
> According to Investopedia, consensus estimates is normally an average or median of all the forecasts from individual analysts tracking a particular stock. In this case, the consensus estimate is for ***EPS for each firm present in the index as of 2019.*** Forecasted EPS is calculated by ***quarterly earnings,*** usually by each firm's fiscal period. Estimates of quarterly earnings are published at the beginning of each quarterly period.

In [12]:
#historic forecasted EPS 
eps_fc = pd.read_csv(PATH + 'sp-eps-fc.csv')

In [13]:
eps_fc.head()

,Term Forecasted,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,1999Q1,NaN,NaN,NaN,0.025,NaN,NaN,-0.10,0.423,NaN,...,0.456,0.090,0.239,0.123,NaN,NaN,0.123,NaN,0.622,NaN
1,1999Q2,NaN,NaN,NaN,0.020,NaN,NaN,-0.13,0.420,NaN,...,0.213,0.090,0.264,0.135,NaN,NaN,0.150,NaN,0.661,NaN
2,1999Q3,NaN,NaN,NaN,0.023,NaN,NaN,-0.11,0.381,NaN,...,0.767,0.099,0.300,0.129,NaN,NaN,0.171,NaN,0.693,NaN
3,1999Q4,NaN,NaN,NaN,0.016,NaN,NaN,NaN,0.432,NaN,...,0.432,0.124,0.386,0.178,NaN,NaN,0.164,NaN,0.667,NaN
4,00Q1,NaN,NaN,NaN,0.032,NaN,NaN,NaN,0.441,NaN,...,0.283,0.143,0.436,0.138,NaN,NaN,0.138,NaN,0.668,NaN


**Historic actual EPS**
> Unlike forecasted EPS, actual EPS are the real numbers denoting Earnings-per-Share for a singular firm. Historic actual EPS will be compared to forecasted EPS to draw correlations and comparisons.

In [14]:
#historic actual EPS
eps_act = pd.read_csv(PATH + 'sp-eps-act.csv')

In [15]:
eps_act.head()

,Quarter,Year,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,Q1,1999,0.16,0.99,NaN,0.040000,NaN,0.0925,-0.030,0.44,...,0.34,0.0000,0.210,0.123333,1.64,NaN,0.1725,NaN,0.56,NaN
1,Q2,1999,0.35,1.76,NaN,0.035357,NaN,0.1000,-0.145,0.42,...,0.06,0.0975,0.285,0.133333,2.28,NaN,0.2900,NaN,0.60,NaN
2,Q3,1999,0.30,1.86,NaN,0.050357,NaN,0.1075,-0.135,0.30,...,0.63,0.0325,0.315,0.130000,1.96,NaN,0.3200,NaN,0.64,NaN
3,Q4,1999,0.32,1.89,NaN,0.024643,NaN,0.0375,-0.080,0.43,...,0.43,0.1300,0.600,0.180000,1.72,NaN,0.2375,NaN,0.49,NaN
4,Q1,2000,0.29,0.89,NaN,0.040585,NaN,0.1050,-0.045,0.45,...,0.45,0.1650,0.500,0.140000,-1.48,NaN,0.2025,NaN,-0.33,NaN


**Historic actual EOD**
> Though this is not directly related to EPS data, EOD would be an interesting measure to use when generating intriguing visualizations and analyses. Who knows what visuals and conclusions I would arrive to with this measure. 

In [16]:
#historic actual EOD
eod_act = pd.read_csv(PATH + 'sp-eod-act.csv')

In [17]:
eod_act.head()

,date,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,3/31/1999,NaN,NaN,NaN,1.2835,NaN,8.2934,6.250,20.9504,NaN,...,NaN,20.2813,35.2813,7.7500,140.6214,NaN,12.6284,NaN,66.5000,NaN
1,6/30/1999,NaN,NaN,NaN,1.6540,NaN,6.1859,6.875,20.3630,NaN,...,NaN,28.6250,38.5625,9.6250,155.6057,NaN,9.7297,NaN,63.5000,NaN
2,9/30/1999,NaN,NaN,NaN,2.2612,NaN,5.7462,7.750,16.4471,NaN,...,NaN,32.7656,37.9688,7.5833,110.4883,NaN,7.3591,NaN,55.1250,NaN
3,12/31/1999,52.0909,NaN,NaN,3.6719,NaN,3.6843,18.375,16.2513,NaN,...,NaN,45.4688,40.2813,7.8750,59.7723,NaN,6.9434,NaN,59.1875,NaN
4,3/31/2000,70.0721,NaN,NaN,4.8504,NaN,3.6388,20.250,15.7478,NaN,...,NaN,82.8125,38.9063,9.4583,68.4994,NaN,5.5839,NaN,41.6250,NaN


**Historic forecasted EPS 3 months prior**
> Instead of using forecast data collected at the beginning of the fiscal period, this feature contains EPS data projected 3 months before the current fiscal period. This is an interesting metric to see how differently forecasters make their predictions at different times.

In [18]:
#historic forecasted EPS 3-months prior
eps_fc_terms = pd.read_csv(PATH + 'sp-eps-fc-terms.csv')

In [19]:
eps_fc_terms.head()

,Forecast Made,Term Forecasted,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,10/1/1999,00Q1,NaN,NaN,NaN,0.030,NaN,NaN,NaN,0.466,...,NaN,0.143,0.398,0.140,NaN,NaN,0.144,NaN,0.777,NaN
1,1/1/2000,00Q2,NaN,NaN,NaN,0.026,NaN,NaN,NaN,0.437,...,NaN,0.168,0.378,0.153,NaN,NaN,0.184,NaN,0.790,NaN
2,4/1/2000,00Q3,NaN,NaN,NaN,0.029,NaN,NaN,NaN,0.414,...,0.84,0.189,0.425,0.149,NaN,NaN,0.218,NaN,0.803,NaN
3,7/1/2000,00Q4,NaN,NaN,NaN,0.032,NaN,NaN,-0.18,0.477,...,0.53,0.211,0.487,0.200,NaN,NaN,0.244,NaN,0.777,NaN
4,9/1/2000,01Q1,NaN,NaN,NaN,0.041,NaN,NaN,-0.09,0.495,...,0.40,0.259,0.475,0.158,NaN,NaN,0.178,NaN,0.750,NaN


In [216]:
eps_fc_terms.shape

(80, 507)

### B) Assess <a id = "assess"></a>

> The following DataFrames contain data for each firm across various dates. To account for all firm averages, my goal is to generate a CSV file where each row contains the firm average, firm type, and divide them into yearly, quarterly, and twenty-year averages.

In [20]:
dict_dfs = {'eps_fc' : eps_fc,
           'eps_actual' : eps_act,
           'eod_actual' : eod_act,
           'eps_fc_terms' : eps_fc_terms}

### Check Missing Data

In [21]:
for key, df in dict_dfs.items():
    print(key, df.shape)

eps_fc (84, 506)
eps_actual (84, 507)
eod_actual (84, 506)
eps_fc_terms (80, 507)


***We need to make sure the number of firms in each DataFrame is consistent.***

---

**Observation 1:** for `eps_fcast`, there are 505 firms encompassing 84 quarterly fiscal periods since 1999.

> There are 506 columns: 1 column being `Term Forecast`, the rest firm names.

**Observation 2:** for `eps_actual`, there are 505 firms encompassing 84 quarterly fiscal periods since 1999.

> There are 507 columns: 2 columns being `Quarter` and `Year`, the rest firm names.

**Observation 3:** for `eod_actual`, there are 505 firms encompassing 84 quarterly calendar periods since 1999.

> There are 506 columns: 1 column being `date`, the rest firm names.

**Observation 4:** for `eps_fcast_terms`, there are 505 firms encompassing 80 quarterly calendar periods since 1999.

> There are 507 columns: 2 columns being `Forecast Made` and `Term Forecasted`, the rest firm names.
 
Since there are only 80 quarterly calendar periods, that ***implies an entire year is missing.***

**Observation 5:** For `eps_fcast`, `eps_actual`, and `eod_actual`, since there are 4 quarters in a year, 84 quarterly forecast periods equate to 21 years. This is correct since we are analyzing the years from 1999 until the end of 2019.

### Most importantly, the number of firms across all DataFrames is consistent.

----

***Check which year is missing from `eps_fcast_terms`***.

> Each **Term Forecasted** entry under `eps_fcast_terms` records the year with 2 digits, so Quarter 1 of the year 2000 becomes 00Q1.
- isolate the first 2 characters to get the year
- join a '20' in front of the string so 00 becomes 2000
- list the number of unique values.

In [22]:
#iterate over all years in eps_fcast_terms, append '20' in front of string
list(map(lambda x: '20' + x, eps_fc_terms['Term Forecasted'].str[:2].unique()))

['2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019']

**Observation**: The year 1999 is missing from `eps_fc_terms`. This makes sense because the start of the forecasting period would be in the last quarter of 1999, which is October. 

---

### Check Nulls

In [23]:
for key, df in dict_dfs.items():
    #display (row, column) per DataFrame
    print('# NaN in {}: {}'.format(key, df.isna().sum().sum()))

# NaN in eps_fc: 7055
# NaN in eps_actual: 5021
# NaN in eod_actual: 6921
# NaN in eps_fc_terms: 7080


**Observation:** All four DataSets contain null values.
> In order to combat this, we'll have to look at both the **number of rows** and **number of columns** with missing data, separately. This way, we can isolate which firms and/or time periods contain complete or incomplete data.

In [24]:
#check rows for missing data
for key, df in dict_dfs.items():
    num_rows_missing = df.isna().sum().max()
    print('{} has {} time periods containing missing data, out of {} total rows.'.format(key, num_rows_missing, df.shape[0]))

eps_fc has 82 time periods containing missing data, out of 84 total rows.
eps_actual has 84 time periods containing missing data, out of 84 total rows.
eod_actual has 83 time periods containing missing data, out of 84 total rows.
eps_fc_terms has 80 time periods containing missing data, out of 80 total rows.


In [25]:
len(eps_fc.columns[eps_fc.isnull().any()])

254

In [26]:
#check columns for missing data
for key, df in dict_dfs.items():
    cols_missing = df.columns[df.isnull().any()]
    num_cols_missing = len(cols_missing)
    print('{} has {} firms containing missing data, out of {} total columns.'
         .format(key, num_cols_missing, df.shape[1]))

eps_fc has 254 firms containing missing data, out of 506 total columns.
eps_actual has 432 firms containing missing data, out of 507 total columns.
eod_actual has 505 firms containing missing data, out of 506 total columns.
eps_fc_terms has 340 firms containing missing data, out of 507 total columns.


**Observation 1:** The only two datasets with incomplete data for all time periods are **actual EPS** and **forecasted EPS 3 months prior.**

> To address this problem, it'd be helpful to isolate the time period ranges for the datasets with incomplete data by row, **forecasted EPS** and **actual EOD price.**

**Observation 2:** For all datasets, all firms contain incomplete data across all time periods.
> This is expected, as analyzing financial history spanning over 20 years will naturally be rife with missing and inaccurate data. The ***good news is that `eps_fcast`, `eps_actual`, and `eps_fcast_terms` are the most complete, while `eod_actual` contains the most amount of missing data.***


**Moving forward, we need to make sure that these inconsistencies won't clash with our analysis.**
> **My approach:** instead of looking at rows and columns ***with*** missing data, we'll be looking at rows and columns that ***are all missing data.***

I figured that if there is some missing data here and there scattered throughout the matrix, then that should not skew our analysis too much.

However, if there a significant amount of rows/columns that are entirely empty, then we ***might have to get ready to drop some dates and firms from our data overall.***

In [231]:
#check for empty rows, return False if row contains at least one non-null value, True if all are null
for key, df in dict_dfs.items():
    cols_check = df.columns
    num_empty_rows = (df[cols_check].isnull().apply(lambda x: all(x), axis = 1)).value_counts()
    print(key, '\n', num_empty_rows, '\n----')

eps_fc 
 False    84
dtype: int64 
----
eps_actual 
 False    84
dtype: int64 
----
eod_actual 
 False    84
dtype: int64 
----
eps_fc_terms 
 False    80
dtype: int64 
----


**Observation:** All datasets do not contain empty rows.
> This is good news, since we can rely on the firms' averages per row instead of having to drop or limit time periods.

In [226]:
#check for empty columns, return False if column contains at least one non-null value, True if all are null
for key, df in dict_dfs.items():
    cols_check = df.columns
    num_empty_cols = df[cols_check].isnull().apply(lambda x: all(x), axis = 0).value_counts()
    print(key, '\n', num_empty_cols)

eps_fc 
 False    506
dtype: int64
eps_actual 
 False    506
True       1
dtype: int64
eod_actual 
 False    506
dtype: int64
eps_fc_terms 
 False    506
True       1
dtype: int64


**Observation:** `eps_actual` and `eps_fcast_terms` are the only datasets that have an empty column.
> Let's isolate and look at the singular empty column for both DataFrames.

In [29]:
#helper function to return an array of column names containing empty data
def comb_cols(df):
    empty_cols = []
    for column in df:
        if df[column].isnull().all():
            empty_cols.append(column)
            
    return empty_cols

In [30]:
#comb datasets for empty column names
print('In eps_act, the firm {} has no data.'.format(comb_cols(eps_act)))
print('In eps_fc_terms, the firm {} has no data.'.format(comb_cols(eps_fc_terms)))

In eps_act, the firm ['AMCR UN Equity'] has no data.
In eps_fc_terms, the firm ['AMCR UN Equity'] has no data.


**Observation:** The same firm in both datasets is empty of data.
> Though this is an annoying error to deal with, it still is to our advantage that both datasets ***share one firm*** in common for missing data. This way, we don't have to worry about dropping two entire firms.

---

### Check Duplicate Data

In [232]:
for key, df in dict_dfs.items():
    print(key, df.duplicated().sum())

eps_fc 0
eps_actual 0
eod_actual 0
eps_fc_terms 0


**Observation:** For all datasets, there are ***no duplicate data.*** This is good news!

**Next, I will check for duplicated firm names.** Although the presence of duplicated firm names will inherently imply duplicated data, sometimes data gets dispersed in weird, unexpected ways, especially when dealing with large datasets.

In [32]:
#check for duplicated firm names
for key, df in dict_dfs.items():
    print(key, df.columns.duplicated().sum())

eps_fc 0
eps_actual 0
eod_actual 0
eps_fc_terms 0


**Observation:** For all datasets, there are ***no duplicate firm names.*** This is also good news.

### There is no need to dedupe our data during the cleaning stage.

---

### Check Data Types

> Make sure all numerical data types are consistent.
- EPS and EOD values must all be **floats**
- Dates should all be **objects** (for now)

- `eps_fc` contains ***1 date field,***, so we should expect ***1 object type.***
- `eps_act` contains ***2 date fields,*** so we should expect ***2 object types.***
- `eod_act` contains ***1 date field,*** so we should expect ***1 object type.***
- `eps_fc_terms` contains ***2 date fields,*** so we should expect ***2 object types.***

> And everything else should be a ***float*** type.

**Examine each dataset's overall data types per column.**

In [33]:
for key, df in dict_dfs.items():
    print(key,'\n', df.dtypes.value_counts(), '\n--------')

eps_fc 
 float64    505
object       1
dtype: int64 
--------
eps_actual 
 float64    499
object       7
int64        1
dtype: int64 
--------
eod_actual 
 float64    505
object       1
dtype: int64 
--------
eps_fc_terms 
 float64    505
object       2
dtype: int64 
--------


**Observation 1:** `eps_fc`, `eod_act`, and `eps_fc_terms` are all consistent with the expected number of object types.

> But we'll still have to double check these facts later.

**Observation 2:** `eps_act` contains 5 more object columns than expected.

---

***Isolate 'Object' Columns under `eps_act`***

In [34]:
eps_act.select_dtypes(include = 'object').head()

,Quarter,BRK/B UN Equity,FOX UW Equity,GOOG UW Equity,HCP UN Equity,SYMC UW Equity,UA UN Equity
0,Q1,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
1,Q2,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
2,Q3,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
3,Q4,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
4,Q1,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable


**Observation:** These firms are all 'Object' types because there is no recorded data under them.
> This is actually missing data. From `eps_act`, there are 6 firms that are empty of data.

### This was a tricky situation to spot, but I caught it in the end. There actually is missing data after all, but we missed those 5 entire firms during the "Check Missing Data" stage, because they were recorded as objects. This means that Pandas wrongly recognized these 5 firms as "complete."

### Quality

**Missing Data**
> **In the end, I decided not to drop any rows or columns with missing data. For the sake of data preservation, I kept all missing data recorded as NaN.**

-  `eps_fc_terms` is missing the year 1999.
- `eps_act` and `eps_fc_act` have one firm with empty data: 'AMCR UN Equity'
- `eps_act` contains 7 empty firms: BRK/B UN Equity, FOX UW Equity, GOOG UW Equity, HCP UN Equity, SYMC UW Equity, UA UN Equity
--- 
- Unnormalized date formats among all DataFrames.
- Firm names across all DataFrames are capitalized and contain white space.
- Erroneous data type for `eps_act` Object columns.
- Erroneous data type for **date** under `eod_act`
- Erroneous data type for **forecast_made** under `eps_fc_terms`
- Erroneous data types for **term** under `eps_act`, `eps_fc`, and `eps_fc_terms` to DateTime quarter index
- No recorded quarterly data for `eod_act`
---
- Firm names not referenced by **firm_id**
- No recorded 20-year for each dataset.
- No recorded yearly averages for each dataset
- No recorded quarterly averages for each dataset.
---
- Features datasets 

### Tidiness
- Unnormalized data among `df_twenty_year_avgs`, `df_yearly_avgs`, and `df_quarter_avgs`
- 20-year, yearly, and quarterly averages contained in different DataFrames.
- Unnormalized data among `eps_act`, `eps_fc`, `eod_act`, and `eps_fc_terms`


### C) CLEAN <a id="clean"></a>

In [35]:
eps_fc_clean = eps_fc.copy()
eps_act_clean = eps_act.copy()
eod_act_clean = eod_act.copy()
eps_fc_terms_clean = eps_fc_terms.copy()

In [36]:
#create dictionary of all clean dfs for iteration
dict_clean = {'eps_fc_clean' : eps_fc_clean,
             'eps_act_clean' : eps_act_clean,
             'eod_act_clean' : eod_act_clean,
             'eps_fc_terms_clean' : eps_fc_terms_clean}

### Code
> **ISSUE 1:**  Unnormalized date formats among all DataFrames.

**Define:** 
- Conjoin `eps_act_clean` dates from 2 columns into 1 to match `eps_fc_clean` format.
> The format we want is YYYYQN. For example, Quarter 1 in 2005 will be 2005Q1.

- Rename both fiscal periods under `eps_act_clean` and `eps_fc_clean` to **term.**

- For both fiscal periods under `eps_fc_terms_clean`, add an underscore.
- Under `eps_fc_terms_clean`, format **term** to YYYYQN.

In [37]:
#create new column, combine year and quarter into 1 string
eps_act_clean['term'] = eps_act_clean['Year'].map(str) + eps_act_clean['Quarter']

In [38]:
#drop 'Year' and 'Quarter'
eps_act_clean.drop(['Year', 'Quarter'], axis = 1, inplace = True)

In [39]:
#designate new column as first column
reorder_cols = eps_act_clean.columns.tolist()
reorder_cols.insert(0, reorder_cols.pop(reorder_cols.index('term')))

eps_act_clean = eps_act_clean.reindex(columns = reorder_cols)

In [40]:
#rename eps_fc_clean fiscal period to term
eps_fc_clean.rename(index = str, columns = {'Term Forecasted' : 'term'}, inplace = True)

#reassign to dictionary
dict_clean['eps_act_clean'] = eps_act_clean

In [41]:
eps_fc_clean[4:].term = '20' + eps_fc_clean[4:].term.astype(str)

D:\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [42]:
#add an underscore for both date periods under eps_fc_terms_clean
eps_fc_terms_clean.rename(index = str, columns = {'Forecast Made' : 'forecast_made',
                                                 'Term Forecasted' : 'term'},
                         inplace = True)

In [43]:
#under term, add a '20' before each string
eps_fc_terms_clean.term = '20' + eps_fc_terms_clean.term.astype(str)

### Test

In [44]:
#give overview of both DataFrames
eps_fc_clean.sample(3)

,term,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
36,2008Q1,0.393,-1.307,0.786,0.232,NaN,0.313,-0.205,0.623,0.557,...,0.316,0.301,2.148,0.429,NaN,NaN,0.398,1.035,1.01,NaN
69,2016Q2,0.389,1.678,2.111,1.998,1.200,1.585,0.155,0.533,1.184,...,0.404,0.469,0.637,0.700,0.981,0.474,0.739,1.968,NaN,0.444
62,2014Q3,0.741,1.634,1.880,1.232,0.773,0.922,0.140,0.598,1.210,...,0.755,0.545,1.708,0.595,NaN,0.520,0.830,1.301,NaN,0.370


In [45]:
eps_act_clean.sample(3)

,term,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
68,2016Q1,0.37,1.15,2.16,3.300000,0.83,1.60,0.21,0.21,1.31,...,0.47,0.57,0.43,0.72,0.12,0.370000,0.88,0.54,0.38,0.41
76,2018Q1,-0.99,0.34,1.85,3.920000,1.74,3.95,0.85,0.24,1.82,...,0.57,0.63,1.09,0.36,0.08,0.440000,1.30,0.86,1.16,0.72
59,2013Q4,0.64,-5.38,0.68,1.187143,0.70,0.22,0.10,0.37,1.04,...,0.30,0.50,1.91,0.52,1.00,0.368964,0.72,1.38,-0.32,0.21


In [46]:
#check consistent format between both `term` fields
forecasts = pd.concat([eps_fc_clean.term, eps_act_clean.term])

#count number of instances where unique term values are the same
(forecasts.value_counts().sort_index() == 2).value_counts()

True    84
Name: term, dtype: int64

In [47]:
#check that dates are the first columns
for key, df in dict_clean.items():
    print(df.columns[:2])

Index(['term', 'A UN Equity'], dtype='object')
Index(['term', 'A UN Equity'], dtype='object')
Index(['date', 'A UN Equity'], dtype='object')
Index(['forecast_made', 'term'], dtype='object')


## The **term** column under `eps_fc_clean`, `eps_act_clean`, and `eps_fc_terms_clean` are all normalized to YYYQN format. This means `eod_act_clean` is the only one with a different date formatting rule.

### Code
> **ISSUE 2:** Firm names across all DataFrames are capitalized and contain white space.

**Define:** Iterate across all DataFrames. Get rid of everything from the first whitespace character onward. Lowercase column names.

In [48]:
#lowercase names
for key, df in dict_clean.items():
    lower_cols = [x.lower() for x in df.columns]
    df.columns = lower_cols

In [49]:
#get rid of white space
for key, df in dict_clean.items():
    stock_symbols = [x.split(' ', 1)[0] for x in df.columns]
    df.columns = stock_symbols

### Test

In [50]:
#check that all firm names are consistent, should return 505 firm results
all_ticks = []
for key, df in dict_clean.items():
    firm_ticks = np.array(df.columns.values)
    all_ticks.append(firm_ticks)
    
#flatten array
all_ticks = np.hstack(all_ticks)

#assign tick names to Pandas Series
all_ticks = pd.Series(all_ticks)

In [51]:
#look at lowest 3 value counts, SHOULD BE DATE PERIODS
all_ticks.value_counts().sort_values()[:5]

date             1
forecast_made    1
term             3
ntrs             4
nvr              4
dtype: int64

In [52]:
#look at highest value counts, should be FOUR (because there are 4 dataframes all with firm names)
all_ticks.value_counts().sort_values(ascending = False)[:5]

alxn    4
are     4
ben     4
syy     4
snps    4
dtype: int64

In [53]:
#check for any upper case instance among columns
all_ticks.str.isupper().any()

False

In [54]:
#check for any white space
all_ticks.str.contains(' ').any()

False

### Code
> **ISSUE 3:** Erroneous data type for `eps_act` Object columns.

**Define:**

- Convert '#N/A Field Not Applicable' strings into 'NaN'
- Convert the 6 `eps_act_clean` Object columns into NaN type.

In [55]:
#isolate 6 firm names recorded as 'object'
obj_ticks = eps_act_clean.select_dtypes(include = 'object').columns.values

#exclude 'term'
obj_ticks = obj_ticks[1:]

In [56]:
#convert to NaN
str_replace = '#N/A Field Not Applicable'
eps_act_clean[obj_ticks] = eps_act_clean[obj_ticks].replace(str_replace, np.nan)

### Test

In [57]:
eps_act_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Columns: 506 entries, term to zts
dtypes: float64(505), object(1)
memory usage: 332.1+ KB


In [58]:
#make sure 'term' is the only object-type column
eps_act_clean.select_dtypes(include = 'object').columns.values.tolist()

['term']

### Code

> **ISSUE 4:**  Erroneous data type for **date** under `eod_act`

**Define:** Convert column `date` to DateTime object.

In [59]:
eod_act_clean.date = pd.to_datetime(eod_act_clean.date, infer_datetime_format=True)

### Test

In [60]:
eod_act_clean.sample(2)

,date,a,aal,aap,aapl,abbv,abc,abmd,abt,acn,...,xel,xlnx,xom,xray,xrx,xyl,yum,zbh,zion,zts
14,2002-09-30,8.7994,NaN,17.58,1.0357,NaN,17.3254,3.350,18.0806,14.28,...,NaN,15.838,31.90,20.085,13.0412,NaN,9.9625,38.34,43.53,NaN
13,2002-06-28,15.9347,NaN,18.17,1.2657,NaN,18.4364,8.479,16.8498,19.00,...,NaN,22.430,40.92,18.455,18.3631,NaN,10.5162,35.66,52.10,NaN


In [61]:
#confirm datetime64 present
eod_act_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Columns: 506 entries, date to zts
dtypes: datetime64[ns](1), float64(505)
memory usage: 332.1 KB


### Code
> **ISSUE 5:** Erroneous data type for **forecast_made** under `eps_fc_terms`.

**Define:**

- Convert column **forecast_made** to DateTime object.

In [62]:
eps_fc_terms_clean.forecast_made = pd.to_datetime(eps_fc_terms_clean.forecast_made, infer_datetime_format=True)

### Test

In [63]:
#confirm presence of datetime64 object
eps_fc_terms_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 0 to 79
Columns: 507 entries, forecast_made to zts
dtypes: datetime64[ns](1), float64(505), object(1)
memory usage: 317.5+ KB


In [64]:
eps_fc_terms_clean.sample(2)

,forecast_made,term,a,aal,aap,aapl,abbv,abc,abmd,abt,...,xel,xlnx,xom,xray,xrx,xyl,yum,zbh,zion,zts
59,2014-07-01,2014Q4,0.937,1.178,1.447,1.334,0.892,0.964,0.063,0.697,...,0.305,0.555,1.906,0.649,1.244,0.629,0.999,1.752,0.454,0.391
48,2011-10-01,2012Q1,0.697,-1.200,1.510,1.259,NaN,0.612,-0.057,1.038,...,0.490,0.530,2.245,0.533,1.001,NaN,0.713,1.298,0.367,NaN


### Code
> **ISSUE 6:** Erroneous data types for **term** under `eps_act`, `eps_fc`, and `eps_fc_terms` to DateTime quarter index

**Define:** Convert YYYYQQ formats into DateTime quarter index type

In [65]:
#helper function to turn a column into Quarter Period type
def convert_period(df, col):
    return pd.to_datetime(df[col]).dt.to_period('Q')
# eps_fc_terms_clean.term = pd.PeriodIndex(eps_fc_terms_clean.term, freq='Q')

In [66]:
#convert term fields to Period type
eps_fc_clean.term = convert_period(eps_fc_clean, 'term')
eps_act_clean.term = convert_period(eps_act_clean, 'term')
eps_fc_terms_clean.term = convert_period(eps_fc_terms_clean, 'term')

### Test

In [67]:
#verify Period Object types
print(eps_fc_clean.dtypes['term'])
print(eps_act_clean.dtypes['term'])
print(eps_fc_terms_clean.dtypes['term'])

period[Q-DEC]
period[Q-DEC]
period[Q-DEC]


### Code
> **ISSUE 7:** No recorded quarterly data for `eod_act`

**Define:** Add a new column **term**, which extracts the *year* and *quarter* from the **date** column

In [68]:
eod_act_clean['term'] = eod_act_clean.date.dt.to_period('Q')

In [69]:
#reorder columns
eod_cols = ['date', 'term'] + [col for col in eod_act_clean.iloc[:, 1:] if col!='term']

In [70]:
eod_act_clean = eod_act_clean.reindex(columns = eod_cols)

### Test

In [71]:
#verify column order
eod_act_clean.head()

,date,term,a,aal,aap,aapl,abbv,abc,abmd,abt,...,xel,xlnx,xom,xray,xrx,xyl,yum,zbh,zion,zts
0,1999-03-31,1999Q1,NaN,NaN,NaN,1.2835,NaN,8.2934,6.250,20.9504,...,NaN,20.2813,35.2813,7.7500,140.6214,NaN,12.6284,NaN,66.5000,NaN
1,1999-06-30,1999Q2,NaN,NaN,NaN,1.6540,NaN,6.1859,6.875,20.3630,...,NaN,28.6250,38.5625,9.6250,155.6057,NaN,9.7297,NaN,63.5000,NaN
2,1999-09-30,1999Q3,NaN,NaN,NaN,2.2612,NaN,5.7462,7.750,16.4471,...,NaN,32.7656,37.9688,7.5833,110.4883,NaN,7.3591,NaN,55.1250,NaN
3,1999-12-31,1999Q4,52.0909,NaN,NaN,3.6719,NaN,3.6843,18.375,16.2513,...,NaN,45.4688,40.2813,7.8750,59.7723,NaN,6.9434,NaN,59.1875,NaN
4,2000-03-31,2000Q1,70.0721,NaN,NaN,4.8504,NaN,3.6388,20.250,15.7478,...,NaN,82.8125,38.9063,9.4583,68.4994,NaN,5.5839,NaN,41.6250,NaN


In [72]:
#verify 84 columns
eod_act_clean.shape[0]

84

In [73]:
eod_act_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Columns: 507 entries, date to zts
dtypes: datetime64[ns](1), float64(505), period[Q-DEC](1)
memory usage: 332.8 KB


## All full years are normalized to YYYY-MM-DD, and all quarterly records to YYYY-QQ.

## This is to enable for more efficient handling, cleaning, and classifying of data later on.

### Code

**ISSUE 8:** Firm names not referenced by **firm_id**

**Define:**
- Assign a **firm_id** to each firm for future normalization.

- Generate new CSV named `firms.csv`

In [74]:
#get firm names
firm_names = eps_act_clean.columns[1:].to_numpy()

In [75]:
#assign ids to firm names
firm_names_ids = {}
counter = 0

for firm in firm_names:
    firm_names_ids.update({firm: counter})
    counter += 1

In [76]:
#generate new DF
firm_ids = pd.DataFrame(list(firm_names_ids.items())).rename(columns = {0: 'firm', 1: 'firm_id'})

In [77]:
#switch firm and firm_id columns
firm_ids = firm_ids.reindex(columns = ['firm_id', 'firm'])

### Test

In [78]:
firm_ids.sample(10)

,firm_id,firm
191,191,fls
395,395,regn
330,330,mtb
385,385,psx
90,90,cdw
0,0,a
350,350,nue
452,452,ua
325,325,mro
173,173,ew


In [79]:
firm_ids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 2 columns):
firm_id    505 non-null int64
firm       505 non-null object
dtypes: int64(1), object(1)
memory usage: 8.0+ KB


### Code
> **ISSUE 9:** No recorded 20-year averages for each dataset.

**Define:**


- Isolate 20-year averages for each firm into its own DataFrame

- Create new DataFrame `twenty_avgs` depicting all 20-year averages for each firm

In [80]:
#convert foreign keys to str temporarily
firm_ids_dict = {str(k):v for k, v in firm_names_ids.items()}

In [81]:
def get_twenty_yr_avgs(df, index_names, col_name, firm_ids_dict):
    df = pd.DataFrame({col_name: df[index_names].mean()}).rename_axis('firm_id').reset_index()
    
    #set 'firm_id' type int64
    df['firm_id'] = df['firm_id'].map(lambda s: firm_ids_dict.get(s) if s in firm_ids_dict else s)
    
    return df

In [82]:
#get 20-year averages
eps_fc_twenty_avg = get_twenty_yr_avgs(eps_fc_clean, firm_names, 'eps_fc', firm_ids_dict)
eps_act_twenty_avg = get_twenty_yr_avgs(eps_act_clean, firm_names, 'eps_act', firm_ids_dict)
eod_act_twenty_avg = get_twenty_yr_avgs(eod_act_clean, firm_names, 'eod_act', firm_ids_dict)
eps_fc_terms_twenty_avg = get_twenty_yr_avgs(eps_fc_terms_clean, firm_names, 'eps_fc_terms', firm_ids_dict)

In [83]:
#put averages into a list
twenty_avg_list = [eps_fc_twenty_avg,
                  eps_act_twenty_avg,
                  eod_act_twenty_avg,
                  eps_fc_terms_twenty_avg]

In [84]:
#merge all Series
df_twenty_year_avgs = reduce(lambda x, y: pd.merge(x, y, on = 'firm_id', how = 'outer'), twenty_avg_list)

### Test

In [85]:
df_twenty_year_avgs.sample(5)

,firm_id,eps_fc,eps_act,eod_act,eps_fc_terms
27,27,0.786224,0.516265,26.256899,0.886074
354,354,0.083833,-0.065484,14.560577,0.095760
139,139,0.586952,0.584727,41.484419,0.631437
14,14,0.202857,0.067798,43.845867,0.206675
488,488,0.233048,0.113171,24.628083,0.288237


In [86]:
df_twenty_year_avgs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505 entries, 0 to 504
Data columns (total 5 columns):
firm_id         505 non-null int64
eps_fc          505 non-null float64
eps_act         498 non-null float64
eod_act         505 non-null float64
eps_fc_terms    504 non-null float64
dtypes: float64(4), int64(1)
memory usage: 23.7 KB


In [87]:
#check there are 505 firms
df_twenty_year_avgs.firm_id.nunique()

505

In [88]:
#check missing data
df_twenty_year_avgs.isna().any()

firm_id         False
eps_fc          False
eps_act          True
eod_act         False
eps_fc_terms     True
dtype: bool

In [89]:
#check duplicate data
df_twenty_year_avgs.duplicated().any()

False

### Code
> **ISSUE 10:** No recorded yearly averages for each dataset.

**Define:**
- Create 4 separate DataFrames for all attributes
- Rename columns to "Feature_Year" (e.g. eps_fc_1999, eod_act_2000, etc.)
- Outer merge all DataFrames to create new DataFrame, `df_yearly_avgs`, on **firms** attribute

In [90]:
#helper function to calculate yearly average of each DataFrame
def calculate_df_yearly_avgs(df, col, firm_ids_dict):
    #extract year from dates, reset index, and remove index name
    df = df.groupby(df[col].dt.year).mean().transpose().rename_axis('firm_id').reset_index()
    
    df['firm_id'] = df['firm_id'].map(lambda s: firm_ids_dict.get(s) if s in firm_ids_dict else s)
    return df

In [91]:
#helper function to melt columns
def melt_avgs(df, firm_id, var_name):
    return df.melt(id_vars = [firm_id],
                value_vars = df.columns.difference([firm_id]),
                var_name = var_name,
                value_name = 'average')

In [92]:
#calculate and melt
def create_df_yearly_avgs(df, col, firm_ids_dict, firm_id):
    df = calculate_df_yearly_avgs(df, 'term', firm_ids_dict)
    df = melt_avgs(df, firm_id, 'year')
    return df

In [93]:
#generate DataFrames of all yearly averages
df_eps_fc_yearly_avgs = create_df_yearly_avgs(eps_fc_clean, 'term', firm_ids_dict, 'firm_id')
df_eps_act_yearly_avgs = create_df_yearly_avgs(eps_act_clean, 'term', firm_ids_dict, 'firm_id')

#for eod_act, doesn't matter if we use date or term field
df_eod_act_yearly_avgs = create_df_yearly_avgs(eod_act_clean, 'date', firm_ids_dict, 'firm_id')
df_eps_fc_terms_yearly_avgs = create_df_yearly_avgs(eps_fc_terms_clean, 'term', firm_ids_dict, 'firm_id')

In [94]:
def store_df(path, df, csv_name):
    df.to_csv(path + csv_name, encoding = 'utf-8', index = False)

In [95]:
#create copies
eps_fc_yearly_avgs = df_eps_fc_yearly_avgs.copy()
eps_act_yearly_avgs = df_eps_act_yearly_avgs.copy()
eod_act_yearly_avgs = df_eod_act_yearly_avgs.copy()
eps_fc_terms_yearly_avgs = df_eps_fc_terms_yearly_avgs.copy()

In [96]:
#add new column entailing "type" of feature before concat
eps_fc_yearly_avgs = eps_fc_yearly_avgs.assign(feature = 'eps_fc')
eps_act_yearly_avgs = eps_act_yearly_avgs.assign(feature = 'eps_act')
eod_act_yearly_avgs = eod_act_yearly_avgs.assign(feature = 'eod_act')
eps_fc_terms_yearly_avgs = eps_fc_terms_yearly_avgs.assign(feature = 'eps_fc_terms')

In [97]:
yearly_avgs_arr = [eps_fc_yearly_avgs, 
                  eps_act_yearly_avgs,
                  eod_act_yearly_avgs,
                  eps_fc_terms_yearly_avgs]

In [98]:
#merge all yearly average DataFrames
df_yearly_avgs = pd.concat(yearly_avgs_arr)

### Test

In [99]:
#each average, individually

#eps_fc_yearly
eps_fc_yearly_avgs.sample(10)

,firm_id,year,average,feature
5326,276,2009,0.08300,eps_fc
1649,134,2002,0.09350,eps_fc
5385,335,2009,0.35725,eps_fc
5654,99,2010,0.98875,eps_fc
0,0,1999,NaN,eps_fc
9019,434,2016,2.73625,eps_fc
6371,311,2011,0.84500,eps_fc
211,211,1999,NaN,eps_fc
4420,380,2007,0.64150,eps_fc
8070,495,2014,0.50175,eps_fc


In [100]:
#eps_fc_yearly
eps_act_yearly_avgs.sample(10)

,firm_id,year,average,feature
7986,411,2014,2.245000,eps_act
2074,54,2003,NaN,eps_act
4043,3,2007,0.144643,eps_act
3177,147,2005,NaN,eps_act
2406,386,2003,0.275000,eps_act
3623,88,2006,0.715000,eps_act
5415,365,2009,0.784935,eps_act
2392,372,2003,0.314375,eps_act
4002,467,2006,0.300625,eps_act
4939,394,2008,0.415000,eps_act


In [101]:
#eod_act_yearly
eod_act_yearly_avgs.sample(10)

,firm_id,year,average,feature
4704,159,2008,32.26750,eod_act
4225,185,2007,66.66250,eod_act
9821,226,2018,230.23500,eod_act
6317,257,2011,37.90000,eod_act
6307,247,2011,NaN,eod_act
1296,286,2001,37.34075,eod_act
225,225,1999,50.84375,eod_act
1211,201,2001,38.56375,eod_act
8414,334,2015,55.38500,eod_act
1349,339,2001,19.36000,eod_act


In [102]:
#eps_fc_terms_yearly
eps_fc_terms_yearly_avgs.sample(10)

,firm_id,year,average,feature
7990,415,2015,0.42625,eps_fc_terms
7514,444,2014,1.08350,eps_fc_terms
4632,87,2009,-0.02850,eps_fc_terms
2261,241,2004,NaN,eps_fc_terms
2758,233,2005,0.37150,eps_fc_terms
3583,48,2007,NaN,eps_fc_terms
8415,335,2016,-0.14125,eps_fc_terms
7375,305,2014,0.27825,eps_fc_terms
3388,358,2006,0.60750,eps_fc_terms
5745,190,2011,0.45325,eps_fc_terms


In [103]:
#verify all columns are consistent
print(eps_fc_yearly_avgs.columns, '\n',
      eps_act_yearly_avgs.columns, '\n',
      eod_act_yearly_avgs.columns, '\n',
      eps_fc_terms_yearly_avgs.columns)

Index(['firm_id', 'year', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'year', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'year', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'year', 'average', 'feature'], dtype='object')


In [104]:
#verify there are 505 firms in each dataframe
print(eps_fc_yearly_avgs.firm_id.nunique(), '\n',
      eps_act_yearly_avgs.firm_id.nunique(), '\n',
      eod_act_yearly_avgs.firm_id.nunique(), '\n',
      eps_fc_terms_yearly_avgs.firm_id.nunique())

505 
 505 
 505 
 505


In [105]:
#all yearly averages
df_yearly_avgs.sample(10)

,firm_id,year,average,feature
2782,257,2004,0.032500,eps_act
595,90,2001,NaN,eps_fc_terms
1166,156,2001,0.020625,eps_act
9226,136,2017,1.112500,eps_act
9568,478,2017,0.060000,eps_act
5015,470,2008,0.706000,eps_fc
457,457,1999,NaN,eps_act
7984,409,2014,0.240000,eps_act
3546,11,2007,0.584500,eps_fc_terms
47,47,1999,3.033225,eod_act


In [106]:
#verify 505 firms in all yearly averages df
df_yearly_avgs.firm_id.nunique()

505

In [107]:
df_yearly_avgs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41915 entries, 0 to 10099
Data columns (total 4 columns):
firm_id    41915 non-null int64
year       41915 non-null object
average    35971 non-null float64
feature    41915 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 1.6+ MB


### Code

> **ISSUE 11:** No recorded quarterly averages for each dataset.

**Define:** 

- Parse `eod_act_clean` dates  by **calendar quarter average** with quarterly data in a new DataFrame.
- Create separate DataFrames containing quarterly averages for `eps_fc_clean`, `eps_act_clean`, and `eps_fc_terms_clean` 
- Rename columns to "Quarter_Year_Feature" (e.g. **q1_eps_fc,** etc.)
- Outer merge all DataFrames into a new df `df_quarterly_avgs` on **firms**

In [108]:
#helper function to calculate quarterly average of each DataFrame
def calculate_df_quarterly_avgs(df, col, firm_ids_dict):
    #extract quarter from Period objects
    df =  df.groupby(df[col].dt.strftime('Q%q')).mean().transpose().rename_axis('firm_id').reset_index()
    df['firm_id'] = df['firm_id'].map(lambda s: firm_ids_dict.get(s) if s in firm_ids_dict else s)
    
    return df

In [109]:
#calculate and melt
def create_df_quarterly_avgs(df, col, firm_ids_dict, firm_id):
    df = calculate_df_quarterly_avgs(df, 'term', firm_ids_dict)
    df = melt_avgs(df, firm_id, 'quarter')
    
    #lowercase quarter column
    df.quarter = df.quarter.str.lower()
    return df

In [110]:
#generate all quarterly average DataFrames
df_eps_fc_quarter_avgs = create_df_quarterly_avgs(eps_fc_clean, 'term', firm_ids_dict, 'firm_id')
df_eps_act_quarter_avgs = create_df_quarterly_avgs(eps_act_clean, 'term', firm_ids_dict, 'firm_id')
df_eod_act_quarter_avgs = create_df_quarterly_avgs(eod_act_clean, 'term', firm_ids_dict, 'firm_id')
df_eps_fc_terms_quarter_avgs = create_df_quarterly_avgs(eps_fc_terms_clean, 'term', firm_ids_dict, 'firm_id')

In [111]:
#create copies
eps_fc_quarter_avgs = df_eps_fc_quarter_avgs.copy()
eps_act_quarter_avgs = df_eps_act_quarter_avgs.copy()
eod_act_quarter_avgs = df_eod_act_quarter_avgs.copy()
eps_fc_terms_quarter_avgs = df_eps_fc_terms_quarter_avgs.copy()

In [112]:
#add new column entailing "type" of feature before concat
eps_fc_quarter_avgs = eps_fc_quarter_avgs.assign(feature = 'eps_fc')
eps_act_quarter_avgs = eps_act_quarter_avgs.assign(feature = 'eps_act')
eod_act_quarter_avgs = eod_act_quarter_avgs.assign(feature = 'eod_act')
eps_fc_terms_quarter_avgs = eps_fc_terms_quarter_avgs.assign(feature = 'eps_fc_terms')

In [113]:
#put DataFrames in array
quarterly_avgs_arr = [eps_fc_quarter_avgs,
                  eps_act_quarter_avgs,
                  eod_act_quarter_avgs,
                  eps_fc_terms_quarter_avgs]

In [114]:
#merge all quarterly average DataFrames
df_quarter_avgs = pd.concat(quarterly_avgs_arr)

### Test

In [115]:
#each quarter individually

#eps_fc_quarter
eps_fc_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
481,481,q1,1.440429,eps_fc
1801,286,q4,1.244619,eps_fc
1701,186,q4,0.873571,eps_fc
1285,275,q3,0.234111,eps_fc
1933,418,q4,0.277286,eps_fc
919,414,q2,0.559238,eps_fc
608,103,q2,0.902190,eps_fc
742,237,q2,0.290286,eps_fc
108,108,q1,0.531214,eps_fc
989,484,q2,0.678524,eps_fc


In [116]:
#eps_act_quarter
eps_act_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
653,148,q2,0.403449,eps_act
1695,180,q4,0.117725,eps_act
1484,474,q3,-0.401429,eps_act
520,15,q2,0.716667,eps_act
1588,73,q4,0.436000,eps_act
343,343,q1,0.079934,eps_act
1730,215,q4,1.523000,eps_act
1579,64,q4,0.762233,eps_act
753,248,q2,0.391667,eps_act
158,158,q1,0.942857,eps_act


In [117]:
#eod_act_quarter
eod_act_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
981,476,q2,41.839700,eod_act
272,272,q1,22.370714,eod_act
1280,270,q3,53.544000,eod_act
1363,353,q3,18.720000,eod_act
1428,418,q3,37.352290,eod_act
1368,358,q3,50.118219,eod_act
1755,240,q4,126.306500,eod_act
1267,257,q3,47.822048,eod_act
252,252,q1,76.920769,eod_act
1702,187,q4,39.786333,eod_act


In [118]:
#eps_fc_terms_quarter
eps_fc_terms_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
1614,99,q4,1.615133,eps_fc_terms
817,312,q2,1.126158,eps_fc_terms
1716,201,q4,1.685650,eps_fc_terms
68,68,q1,0.642684,eps_fc_terms
895,390,q2,0.619400,eps_fc_terms
1166,156,q3,0.373050,eps_fc_terms
1520,5,q4,0.794200,eps_fc_terms
1181,171,q3,2.018200,eps_fc_terms
1498,488,q3,0.256600,eps_fc_terms
66,66,q1,0.186850,eps_fc_terms


In [119]:
#verify that all columns are consistent
print(eps_fc_quarter_avgs.columns, '\n',
      eps_act_quarter_avgs.columns, '\n',
      eod_act_quarter_avgs.columns, '\n',
      eps_fc_terms_quarter_avgs.columns)

Index(['firm_id', 'quarter', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'quarter', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'quarter', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'quarter', 'average', 'feature'], dtype='object')


In [120]:
#verify there are 505 firms in each dataframe
print(eps_fc_quarter_avgs.firm_id.nunique(), '\n',
      eps_act_quarter_avgs.firm_id.nunique(), '\n',
      eod_act_quarter_avgs.firm_id.nunique(), '\n',
      eps_fc_terms_quarter_avgs.firm_id.nunique())

505 
 505 
 505 
 505


In [121]:
#all quarterly averages
df_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
828,323,q2,1.323500,eps_act
1410,400,q3,0.346810,eps_fc
385,385,q1,0.982875,eps_fc
1030,20,q3,512.112467,eod_act
1784,269,q4,0.398450,eps_fc_terms
303,303,q1,0.131545,eps_fc_terms
1747,232,q4,0.231667,eps_act
928,423,q2,56.671371,eod_act
755,250,q2,0.334762,eps_act
491,491,q1,0.368923,eps_fc_terms


In [122]:
#verify 505 firms in df_quarter_avgs
df_quarter_avgs.firm_id.nunique()

505

In [123]:
df_quarter_avgs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8080 entries, 0 to 2019
Data columns (total 4 columns):
firm_id    8080 non-null int64
quarter    8080 non-null object
average    8021 non-null float64
feature    8080 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 315.6+ KB


### Code
>**ISSUE 1:** Unnormalized data among `df_twenty_year_avgs`, `df_yearly_avgs`, and `df_quarter_avgs`

**Define:** 
- Normalize features across all 3 DataFrames. 
- For `df_twenty_year_avgs`, melt **feature column names** (eps_fc, eps_act, eod_act, eps_fc_terms) into a single column called **feature**

Desired output for each DataFrame:


| firm_id     | average  | average_type | time_period   | feature|
|:------------|:-------------:|:-----------:|:---------:|-------:|
|501           | 0.21875         | yearly   | 2002        | eps_fc_terms |
| 378          |   0.059000     |   quarterly | q4        | eps_act  |
| 147           | NaN     | yearly   |   2016      | eod_act  |
| 33           |  0.005067       | quarterly  | q4      | eps_fc |

> ***Apply the above table format to all three DFs, then concatenate to create `all_avgs` .***

**Note:** For `df_twenty_year_avgs`, it would be redundant to put *twenty_year* as the value under both **average_type** and **time_period** columns.

To curb this, I will assign all **time_period** values as NaN for `df_twenty_year_avgs`.

In [124]:
#assign column order
avgs_col_order = ['firm_id', 'average', 'average_type', 'time_period', 'feature']

In [125]:
#create clean copies of dfs
twenty_year_clean = df_twenty_year_avgs.copy()
yearly_clean = df_yearly_avgs.copy()
quarterly_clean = df_quarter_avgs.copy()

- ### `twenty_year_clean`

In [126]:
#isolate feature names
feats_col_names = twenty_year_clean.columns.difference(['firm_id'])

In [127]:
#move eps_fc . . . eps_fc_terms under "average_type"
twenty_year_clean = pd.melt(twenty_year_clean,
        id_vars = ['firm_id'],
        value_vars = feats_col_names,
        var_name = 'feature',
        value_name = 'average')

In [128]:
#add 'feature' and 'time_period' columns
twenty_year_clean = twenty_year_clean.assign(average_type = 'twenty_year', time_period = np.nan)

In [129]:
#reorder columns
twenty_year_clean = twenty_year_clean[avgs_col_order]

### Test

In [130]:
#twenty_year
twenty_year_clean.sample(5)

,firm_id,average,average_type,time_period,feature
1276,266,1.198424,twenty_year,NaN,eps_fc
468,468,49.542500,twenty_year,NaN,eod_act
1636,121,0.366400,twenty_year,NaN,eps_fc_terms
1297,287,0.815134,twenty_year,NaN,eps_fc
1525,10,0.672000,twenty_year,NaN,eps_fc_terms


In [131]:
twenty_year_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020 entries, 0 to 2019
Data columns (total 5 columns):
firm_id         2020 non-null int64
average         2012 non-null float64
average_type    2020 non-null object
time_period     0 non-null float64
feature         2020 non-null object
dtypes: float64(2), int64(1), object(2)
memory usage: 79.0+ KB


In [132]:
#verify 505 firm names
twenty_year_clean.firm_id.nunique()

505

In [133]:
#verify all time_period values are NaN
twenty_year_clean.time_period.unique()

array([nan])

In [134]:
#verify average_type contains only twenty_year
twenty_year_clean.average_type.unique()

array(['twenty_year'], dtype=object)

- ### `yearly_clean`

In [135]:
#assign 'average_type' column
yearly_clean = yearly_clean.assign(average_type = 'yearly')

In [136]:
#rename 'year' column to 'time_period'
yearly_clean.rename(columns = {'year' : 'time_period'}, inplace = True)

In [137]:
#reorder columns
yearly_clean = yearly_clean[avgs_col_order]

### Test

In [138]:
yearly_clean.sample(10)

,firm_id,average,average_type,time_period,feature
373,373,NaN,yearly,1999,eod_act
6202,142,0.352750,yearly,2012,eps_fc_terms
8252,172,NaN,yearly,2016,eps_fc_terms
8533,453,0.135593,yearly,2015,eps_act
8245,165,1.292750,yearly,2016,eps_fc_terms
2164,144,NaN,yearly,2003,eod_act
6772,207,0.982500,yearly,2013,eps_fc_terms
3775,240,84.595000,yearly,2006,eod_act
4062,22,1.256500,yearly,2007,eps_fc
651,146,29.451150,yearly,2000,eod_act


In [139]:
yearly_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41915 entries, 0 to 10099
Data columns (total 5 columns):
firm_id         41915 non-null int64
average         35971 non-null float64
average_type    41915 non-null object
time_period     41915 non-null object
feature         41915 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 1.9+ MB


In [140]:
#verify 505 firms
yearly_clean.firm_id.nunique()

505

In [141]:
#verify 'average_type' contains only 'yearly'
yearly_clean.average_type.unique()

array(['yearly'], dtype=object)

In [142]:
#verify 'feature' contains all features
all(feat in feats_col_names for feat in yearly_clean.feature.unique())

True

In [143]:
#verify there are 21 years recorded (1999 - 2019)
len(yearly_clean.time_period.unique())

21

In [144]:
#examine lowest and highest year
yearly_clean.time_period.min(), yearly_clean.time_period.max()

(1999, 2019)

- ### `quarterly_clean`

In [145]:
#assign 'average_type' column
quarterly_clean = quarterly_clean.assign(average_type = 'quarterly')

In [146]:
#rename 'quarter' column to 'time_period'
quarterly_clean.rename(columns = {'quarter' : 'time_period'}, inplace = True)

In [147]:
#reorder columns
quarterly_clean = quarterly_clean[avgs_col_order]

### Test

In [148]:
quarterly_clean.sample(10)

,firm_id,average,average_type,time_period,feature
1633,118,52.310000,quarterly,q4,eod_act
1726,211,0.463263,quarterly,q4,eps_fc
681,176,0.782667,quarterly,q2,eps_act
692,187,43.366721,quarterly,q2,eod_act
1696,181,1.308125,quarterly,q4,eps_fc
7,7,0.565619,quarterly,q1,eps_fc
126,126,0.440000,quarterly,q1,eps_fc
1792,277,32.133440,quarterly,q4,eod_act
1856,341,0.495800,quarterly,q4,eps_fc_terms
1681,166,0.525500,quarterly,q4,eps_act


In [149]:
quarterly_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8080 entries, 0 to 2019
Data columns (total 5 columns):
firm_id         8080 non-null int64
average         8021 non-null float64
average_type    8080 non-null object
time_period     8080 non-null object
feature         8080 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 378.8+ KB


In [150]:
#verify 505 firms
quarterly_clean.firm_id.nunique()

505

In [151]:
#verify 'average_type' contains only 'quarterly'
quarterly_clean.average_type.unique()

array(['quarterly'], dtype=object)

In [152]:
#verify 'time_period' contains only 'q1' till 'q4'
quarterly_clean.time_period.unique()

array(['q1', 'q2', 'q3', 'q4'], dtype=object)

In [153]:
#verify 'feature' contains all features
all(feat in feats_col_names for feat in quarterly_clean.feature.unique())

True

### Code
>**ISSUE 2:** 20-year, yearly, and quarterly averages contained in different DataFrames.

**Define:** Concatenate the normalized `twenty_year_clean`, `yearly_clean`, and `quarterly_clean` into new CSV **all_avgs.csv**.

In [154]:
#put all average DFs in an array
avgs_array = [twenty_year_clean, yearly_clean, quarterly_clean]

In [155]:
all_avgs = pd.concat(avgs_array)

### Test

In [156]:
all_avgs.sample(10)

,firm_id,average,average_type,time_period,feature
8222,142,0.511000,yearly,2016,eps_fc_terms
8826,241,0.686000,yearly,2016,eps_fc
6807,242,46.966250,yearly,2012,eod_act
8662,77,0.801500,yearly,2016,eps_fc
1077,67,129.920095,quarterly,q3,eod_act
5006,461,0.788250,yearly,2009,eps_fc_terms
7516,446,0.478750,yearly,2013,eps_act
1362,352,6.986250,yearly,2002,eps_fc_terms
7322,252,0.755000,yearly,2013,eps_act
9979,384,1.863000,yearly,2018,eps_fc


In [157]:
all_avgs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52015 entries, 0 to 2019
Data columns (total 5 columns):
firm_id         52015 non-null int64
average         46004 non-null float64
average_type    52015 non-null object
time_period     49995 non-null object
feature         52015 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 2.4+ MB


In [158]:
all_avgs.shape

(52015, 5)

In [159]:
#verify 505 firms, no duplicates
all_avgs.firm_id.nunique()

505

In [160]:
#verify only 3 average_types
all_avgs.average_type.unique()

array(['twenty_year', 'yearly', 'quarterly'], dtype=object)

In [161]:
#verify 4 features
all_avgs.feature.unique()

array(['eod_act', 'eps_act', 'eps_fc', 'eps_fc_terms'], dtype=object)

### Code
>**ISSUE 3:** Unnormalized data among `eps_act`, `eps_fc`, `eod_act`, and `eps_fc_terms`

**Define:**

- Achieve desired `features.csv` output:

| firm_id  | feature     | date      | term         |    value    |
|:---------| :-------: | :-----------------:| :-------------: |----------: |
| 485       |     eps_fc  |  NaT           |        2004A1      |  1.40900      |
| 104       | eps_act     |  NaT       |  2001Q4       | -0.113333 |
| 337     | eod_act      | 2019-03-29      |   2019Q1         |   87.490    |
| 276       | eps_fc_terms | 2009-04-01   | 2009Q3   | 0.009 |

> **date** feature contains the **date column in `eod_act`** and **forecast_made in `eps_fc_terms`**. They are assigned to the same column because both columns are consistent: the same DateTime Object.

> **date** feature will be NaN for `eps_fc` and `eps_act`.

- ### `eps_fc` and `eps_act`

> For both DataFrames, we don't have to worry about handling the **date** column. That will be resolved later when merging all the DataFrames with `eod_act` and `eps_fc_terms`

In [162]:
fc_act_order = ['firm_id', 'feature', 'term', 'value']

In [163]:
#melt firm names
eps_fc_clean = eps_fc_clean.melt (id_vars = ['term'],
                  var_name = 'firm_id')

In [164]:
eps_act_clean = eps_act_clean.melt(id_vars = ['term'],
                                  var_name = 'firm_id')

In [165]:
#reassign all firms to firm_id
eps_fc_clean['firm_id'] = eps_fc_clean['firm_id'].map(firm_ids_dict)
eps_act_clean['firm_id'] = eps_act_clean['firm_id'].map(firm_ids_dict)

In [166]:
#add feature column
eps_fc_clean = eps_fc_clean.assign(feature = 'eps_fc')
eps_act_clean = eps_act_clean.assign(feature = 'eps_act')

In [167]:
#reorder columns
eps_fc_clean = eps_fc_clean[fc_act_order]
eps_act_clean = eps_act_clean[fc_act_order]

### Test

In [168]:
eps_fc_clean.sample(10)

,firm_id,feature,term,value
41191,490,eps_fc,2006Q4,NaN
10572,125,eps_fc,2017Q1,0.837
23574,280,eps_fc,2012Q3,0.839
37262,443,eps_fc,2011Q3,0.386
39954,475,eps_fc,2012Q3,0.280
28909,344,eps_fc,2002Q2,0.773
38937,463,eps_fc,2010Q2,0.387
12143,144,eps_fc,2010Q4,0.208
30291,360,eps_fc,2011Q4,0.857
29262,348,eps_fc,2006Q3,0.206


In [169]:
eps_fc_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42420 entries, 0 to 42419
Data columns (total 4 columns):
firm_id    42420 non-null int64
feature    42420 non-null object
term       42420 non-null period[Q-DEC]
value      35365 non-null float64
dtypes: float64(1), int64(1), object(1), period[Q-DEC](1)
memory usage: 1.3+ MB


In [170]:
#verify 505 firms in both
eps_fc_clean.firm_id.nunique() == 505 & eps_act_clean.firm_id.nunique() == 505

True

In [171]:
#examine 'feature' in both
len(eps_fc_clean.feature.unique()) == 1 & len(eps_act_clean.feature.unique()) == 1

True

In [172]:
#verify 84 unique term values: 21 years x 4 quarters
len(eps_fc_clean.term.unique()), len(eps_act_clean.term.unique())

(84, 84)

- ### `eod_act`

In [173]:
#define column order
feats_col_order = ['firm_id', 'feature', 'date', 'term', 'value']

In [174]:
#melt firm ids
eod_act_clean = eod_act_clean.melt(id_vars = ['date', 'term'],
                  var_name = 'firm_id')

In [175]:
#reassign firm_ids
eod_act_clean['firm_id'] = eod_act_clean['firm_id'].map(firm_ids_dict)

In [176]:
#add feature column
eod_act_clean = eod_act_clean.assign(feature = 'eod_act')

In [177]:
#reorder columns 
eod_act_clean = eod_act_clean[feats_col_order]

### Test

In [178]:
eod_act_clean.sample(5)

,firm_id,feature,date,term,value
7277,86,eod_act,2012-06-29,2012Q2,32.7800
39502,470,eod_act,2004-09-30,2004Q3,50.9500
701,8,eod_act,2006-06-30,2006Q2,28.3200
14196,169,eod_act,1999-03-31,1999Q1,291.5625
18284,217,eod_act,2013-03-28,2013Q1,43.9400


In [179]:
eod_act_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42420 entries, 0 to 42419
Data columns (total 5 columns):
firm_id    42420 non-null int64
feature    42420 non-null object
date       42420 non-null datetime64[ns]
term       42420 non-null period[Q-DEC]
value      35499 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(1), object(1), period[Q-DEC](1)
memory usage: 1.6+ MB


In [180]:
#verify 505 firms 
eod_act_clean.firm_id.nunique() == 505 

True

In [181]:
#examine 'feature' column
eod_act_clean.feature.unique()

array(['eod_act'], dtype=object)

In [182]:
#verify 84 unique dates and terms
len(eod_act_clean.term.unique()) == 84, len(eod_act_clean.date.unique()) == 84

(True, True)

- ### `eps_fc_terms`

In [183]:
#melt firm ids
eps_fc_terms_clean = eps_fc_terms_clean.melt(id_vars = ['forecast_made', 'term'],
                       var_name = 'firm_id')

In [184]:
#reassign firm ids
eps_fc_terms_clean['firm_id'] = eps_fc_terms_clean['firm_id'].map(firm_ids_dict)

In [185]:
#rename forecast_made to date
eps_fc_terms_clean = eps_fc_terms_clean.rename(columns = {'forecast_made' : 'date'})

In [186]:
#add feature column
eps_fc_terms_clean = eps_fc_terms_clean.assign(feature = 'eps_fc_terms')

In [187]:
#reorder columns
eps_fc_terms_clean = eps_fc_terms_clean[feats_col_order]

### Test

In [188]:
eps_fc_terms_clean.sample(10)

,firm_id,feature,date,term,value
12566,157,eps_fc_terms,2001-04-01,2001Q3,NaN
6242,78,eps_fc_terms,2000-04-01,2000Q3,NaN
6893,86,eps_fc_terms,2003-01-01,2003Q2,NaN
8928,111,eps_fc_terms,2011-10-01,2012Q1,1.531
38726,484,eps_fc_terms,2001-04-01,2001Q3,0.360
14605,182,eps_fc_terms,2011-01-01,2011Q2,NaN
36864,460,eps_fc_terms,2015-10-01,2016Q1,1.442
16887,211,eps_fc_terms,2001-07-01,2001Q4,0.050
11579,144,eps_fc_terms,2014-07-01,2014Q4,0.218
27860,348,eps_fc_terms,2004-10-01,2005Q1,0.126


In [189]:
#verify 505 firms
eps_fc_terms_clean.firm_id.nunique()

505

In [190]:
#examine 'feature' column
eps_fc_terms_clean.feature.unique()

array(['eps_fc_terms'], dtype=object)

In [191]:
#verify 80 unique terms (1999 not recorded)
eps_fc_terms_clean.term.nunique() == 80

True

- ### `features.csv`

Now's the time to bring everything together!

In [192]:
feat_array = [eps_fc_clean, eps_act_clean, eod_act_clean, eps_fc_terms_clean]

In [193]:
#concat to prepare for outer left merge
eod_act_eps_terms = pd.concat([eod_act_clean, eps_fc_terms_clean], join = 'outer')

In [194]:
#outer join
features = pd.concat([eod_act_eps_terms, eps_fc_clean], join = 'outer', sort = False)
features = pd.concat([features, eps_act_clean], join = 'outer', sort = False)

In [195]:
features.sample(10)

,firm_id,feature,date,term,value
20114,239,eod_act,2008-09-30,2008Q3,41.2000
6039,75,eps_fc_terms,2009-07-01,2009Q4,0.8170
16686,208,eps_fc_terms,2011-04-01,2011Q3,4.2750
34534,431,eps_fc_terms,2013-04-01,2013Q3,0.4350
2766,32,eod_act,2018-09-28,2018Q3,NaN
11616,138,eps_act,NaT,2005Q1,0.5775
6896,82,eps_fc,NaT,2001Q1,0.2540
12549,149,eps_act,NaT,2007Q2,0.4200
35031,437,eps_fc_terms,2017-07-01,2017Q4,1.5080
33247,415,eps_fc_terms,2011-07-01,2011Q4,0.1330


### Test

In [196]:
features.sample(5)

,firm_id,feature,date,term,value
30813,385,eps_fc_terms,2003-01-01,2003Q2,NaN
40536,482,eps_act,NaT,2011Q1,0.730
5098,60,eod_act,2013-09-30,2013Q3,13.800
23801,297,eps_fc_terms,2010-01-01,2010Q2,0.074
5203,65,eps_fc_terms,2000-07-01,2000Q4,0.192


In [197]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167660 entries, 0 to 42419
Data columns (total 5 columns):
firm_id    167660 non-null int64
feature    167660 non-null object
date       82820 non-null datetime64[ns]
term       167660 non-null period[Q-DEC]
value      141079 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(1), object(1), period[Q-DEC](1)
memory usage: 7.7+ MB


In [198]:
#verify 505 terms
features.firm_id.nunique() == 505

True

In [199]:
#examine 'feature' column
features.feature.unique()

array(['eod_act', 'eps_fc_terms', 'eps_fc', 'eps_act'], dtype=object)

In [200]:
#verify 84 term ranges
features.term.nunique() == 84, 

(True,)

In [201]:
#verify 164 date values: 84 ones from eod_act, and 80 from eps_fc_terms
features.date.nunique()

164

# III) Store Data <a id = "store"></a>

**Disclaimer:** I decided to put the following CSVs under the ***./data/clean/averages/components*** directory because they form *part* of the main CSVs under the ***./data/clean/averages*** directory:
- `quarter-avgs-eod-act.csv`
- `quarter-avgs-eps-act.csv`
- `quarter-avgs-eps-fc.csv`
- `quarter-avgs-eps-fc-terms.csv`

> The above are all components of the `quarter-avgs.csv` under **/averages/**

- `yearly-avgs-eod-act.csv`
- `yearly-avgs-eps-act.csv`
- `yearly-avgs-eps-fc.csv`
- `yearly-avgs-eps-fc-terms.csv`

> The above are all components of the `yearly-avgs.csv` under **/averages/**

**Additionally, I decided to keep `avgs.csv` under the *./data/clean/averages/* directory.** 

## Firms

In [202]:
firm_ids.to_csv(PATH_CLEAN + 'firms.csv', encoding = 'utf-8', index = False)

## Averages

In [203]:
#twenty-year-averages
df_twenty_year_avgs.to_csv(PATH_CLEAN_AVGS + 'twenty-year-avgs.csv', encoding = 'utf-8', index = False)

In [204]:
#yearly averagas
df_yearly_avgs.to_csv(PATH_CLEAN_AVGS + 'yearly-avgs.csv', encoding = 'utf-8', index = False)

In [205]:
#quarterly averages
df_quarter_avgs.to_csv(PATH_CLEAN_AVGS + 'quarter-avgs.csv', encoding = 'utf-8', index = False)

In [206]:
#all averages
all_avgs.to_csv(PATH_CLEAN_AVGS + 'avgs.csv', encoding = 'utf-8', index = False)

**NOTE:** For the twenty year, yearly, and quarterly datasets, I stored their old `df_` version instead of their new clean versions. 
> This is because their old versions are stored more efficiently as ***standalone*** CSVs, and were only modified to fit the larger `all_avgs` DataFrame.

### Components

In [207]:
#quarterly averages
store_df(PATH_CLEAN_AVGS_SUB, df_eps_fc_quarter_avgs, 'quarter-avgs-eps-fc.csv')
store_df(PATH_CLEAN_AVGS_SUB, df_eps_act_quarter_avgs, 'quarter-avgs-eps-act.csv')
store_df(PATH_CLEAN_AVGS_SUB, df_eod_act_quarter_avgs, 'quarter-avgs-eod-act.csv')
store_df(PATH_CLEAN_AVGS_SUB, df_eps_fc_terms_quarter_avgs, 'quarter-avgs-eps-fc-terms.csv')

In [208]:
#yearly averages
store_df(PATH_CLEAN_AVGS_SUB, df_eps_fc_yearly_avgs, 'yearly-avgs-eps-fc.csv')
store_df(PATH_CLEAN_AVGS_SUB, df_eps_act_yearly_avgs, 'yearly-avgs-eps-act.csv')
store_df(PATH_CLEAN_AVGS_SUB, df_eod_act_yearly_avgs, 'yearly-avgs-eod-act.csv')
store_df(PATH_CLEAN_AVGS_SUB, df_eps_fc_terms_yearly_avgs, 'yearly-avgs-eps-fc-terms.csv')

## Features

In [209]:
eps_fc_clean.to_csv(PATH_CLEAN + 'eps-fc.csv')
eps_act_clean.to_csv(PATH_CLEAN + 'eps-act.csv')
eod_act_clean.to_csv(PATH_CLEAN + 'eod-act.csv')
eps_fc_terms_clean.to_csv(PATH_CLEAN + 'eps-fc-terms.csv')

features.to_csv(PATH_CLEAN + 'features.csv', index = False)

# IV) Notes


- When dealing with the CSVs depicting averages, we need to keep in mind **missing data** in our interpretations.    
- Keep in mind that `eod_act` and `eps_fc_terms` is based on **calendar years** not fiscal years.

In [210]:
#convert notebook to HTML
from subprocess import call
call(['python', '-m', 'nbconvert', 'data_cleaning.ipynb'])

0